# Import Libraries

In [1]:
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
import openpyxl
import xlsxwriter
import pandas as pd
import time
import datetime
from selenium.webdriver.common.keys import Keys


# SCRAPER

In [46]:
trek_list = pd.read_html('https://www.bikatadventures.com/Home/Blog/Bikat-Rating-Scale')
trek_list = pd.DataFrame(trek_list[0])
Diff_link = []
NA_treks = []
URL = []

URL generator

In [56]:
trek_name2 = trek_list['Activity Name'].unique()
trek_name = []
driver = webdriver.Chrome()
for i in range(0,len(trek_name2)):
    trek_name.append(trek_name2[i].replace(" ","%20"))
for i in trek_name:
    url = 'https://www.bikatadventures.com/Home/Search?searchstring='+i
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        link = 'https://www.bikatadventures.com'+soup.find('div', {'class' : 'col-lg-3 col-md-3 col-sm-4 col-xs-12'}).a['href']
        if i[0:3].lower() in link.lower():
            URL.append(link)
        else:
            Diff_link.append((link,i))
    except AttributeError:
        NA_treks.append(i)

Manual correction for NA and Diff_link

Main Scraper

In [ ]:
driver = webdriver.Chrome()
for i in URL:
    driver.get(i)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')     
    try:
        header = soup.find('div', {'class': 'top-info-card__heading'})
        trek_name = header.h1.text
        region = header.h3.span.text
        no_of_days = header.h3.text
        no_of_days = no_of_days.split(" ")[4]
        header2 = soup.find('div', {'class': 'top-info-card__difficulty'})
        #difficulty = soup.find('div', {'class': 'row'})
        #difficulty = soup.find('div', {'class': 'gauge-wrap'})
        #difficulty = soup.find('div', {'id' : 'g0'})
        difficulty  = soup.find('div', {'style' :'display:none'}).text
        distance = soup.find('div', {'class': 'icon-section'}).text.split("\n")[2]
        altitude = soup.find('div', {'class': 'top-info-card__difficulty--altitude'}).text.split("\n")[2]
        price = soup.find('span', {'class' : 'main-price'}).text
        SD = soup.find('p', {'class' : 'destination-info'}).text
        SD.split("to")
        B = soup.find('div', {'class' : 'col-xs-12 col-lg-9 side accordion-mob'})
        description = B.find('div', {'class' : 'brief-section section para'}).text
        BB = soup.find('div', {'class' : 'form smart-form-new'}).text
        Extra_cost = BB.strip()
        #BB = BB.split("                          ")
        #Single_tent_acc = BB[2].split("\n")[0].strip()
        #Backpack_offloading = BB[3].split("\n")[0].strip()
        #insurance = BB[5].split("\n")[0].strip()
        #travel = BB[4].split("\n")[0].strip()
        #startbase = travel.split(" ")[3]
        #endbase = travel.split(" ")[5]
        A = (trek_name,region,no_of_days,difficulty,distance,altitude,price,SD,description, Extra_cost)
        Data.append(A)
        print(180-len(Data)-len(x))
    except AttributeError:
        x.append(driver.current_url)
        print(180-len(Data)-len(x))
    Main_data=pd.DataFrame(Data, columns=['trek_name','region','no_of_days','difficulty','distance','altitude','price','SD','description', 'Extra_cost'])
    Main_data.to_csv(r'C:\Users\Samarth\Desktop\Main_data.csv')   

## Data Cleaning

Correcting Column names and data types

In [24]:
Main_data.rename(columns = {'altitude':'Altitude_(in_mts)', 'distance' : 'Distance_(in_km)', 'price' : 'price_(in_Rs)'}, inplace = True)
Main_data['Altitude_(in_mts)']=Main_data['Altitude_(in_mts)'].apply(lambda x: int(x.split(" ")[0]))
Main_data['Distance_(in_km)']=Main_data['Distance_(in_km)'].apply(lambda x: int(x.split(" ")[0]))

Main_data['Single_tent']=""
Main_data['Backpack_offloading']=""
Main_data['Travel']=""
Main_data['Insurance']=""

STripping Extracost Column

In [25]:
def stripper(a):
    a = a.replace(" ","")
    a = a.replace("\n","").split("+")
    return(a)
Main_data['Extra_cost'] = Main_data['Extra_cost'].apply(lambda x: stripper(x))

Filling values for extra cost Bifurcation

In [27]:
for i in range (1,77):
    x = Main_data['Extra_cost'][i]
    for z in x: 
        if "Tent" in z:
            Main_data['Single_tent'][i]='Single_tent'+" "+z.split("S")[0]
        elif "Backpack" in z:
            Main_data['Backpack_offloading'][i]="backpack"+" "+z.split("B")[0]
        elif "-" in z:
            Main_data['Travel'][i]=z
        elif "Insurance" in z:
            Main_data['Insurance'][i]="Insurance"+" "+z.split("M")[0]

<ipython-input-27-528cbccb028d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_data['Single_tent'][i]='Single_tent'+" "+z.split("S")[0]
<ipython-input-27-528cbccb028d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_data['Backpack_offloading'][i]="backpack"+" "+z.split("B")[0]
<ipython-input-27-528cbccb028d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_data['Travel'][i]=z
<ipython-input-27-528cbccb028d>:11: Settin

Covert the other columns to digitis

In [28]:
def convdig(x):
    emp_str = ""
    for m in x:
        if m.isdigit():
            emp_str = emp_str + m
    return(emp_str)

Main_data['Insurance']=Main_data['Insurance'].apply(lambda x: convdig(x))
Main_data['Single_tent']=Main_data['Single_tent'].apply(lambda x: convdig(x))
Main_data['Backpack_offloading']=Main_data['Backpack_offloading'].apply(lambda x: convdig(x))


## Merge for Months

Make Manual corrections for unmatched Data

In [38]:
Z = pd.merge(Main_data, trek_list, how='outer', left_on='trek_name', right_on='Activity Name')

Enter the Budget as int  35000


In [64]:
Z['Typical Season']=Z['Typical Season'].apply(lambda x: x.split(","))
Z.explode('Typical Season')
Z["Typical Season"].replace({'Jan':1, 'Feb':2, 'Mar' :3, 'Nov':11, 'Dec' :12, 'May' :5, 'Sep':9, 'Oct':10, 'Jun':6,
       'Aug' : 8, 'Apr' :4, 'Sept' :9, 'Jul' : 7, 'June' :6, ' Aug' : 8, ' Sep' : 9, ' Oct' : 10,
       'July' : 7}, inplace=True)

In [17]:
Final_data.to_pickle('Final_data')

### Data Cleaning through excel

In [2]:
Final_data = pd.read_pickle('Final_data')

In [ ]:
Pass = {'Ali Bedni Bugyal Trek' : 'Yes',
'Bali Pass Trek': 'Yes',
'Beas Kund Trek': 'Yes',
'Bhrigu Lake Trek': 'Yes',
'Brahmatal Trek ': 'Yes',
'Dayara Bugyal Trek': 'Yes',
'Deo Tibba Base Camp Trek': 'Yes',
'Dev Kyara Trek': 'Yes',
'Ghepan Ghat Lake Trek': 'Yes',
'Hampta Pass Trek': 'Yes',
'Har ki Dun Trek': 'Yes',
'Kalihani Pass Trek': 'Yes',
'Kashmir Great Lakes Trek': 'Yes',
'Kedarkantha Trek': 'Yes',
'Mukta Top Trek': 'Yes',
'Nag Tibba Trek': 'Yes',
'Phulara Ridge Trek': 'Yes',
'Rupin Pass Trek': 'Yes',
'Sham Valley Trek': 'Yes',
'Tarsar Marsar Trek': 'Yes'}
Final_data['Pass'] = Final_data['trek_name'].map(Pass).fillna("No")


In [ ]:
Final Required 

In [3]:
x = int(input("No of Days  ")) #no of days
y = int(input("Treks Completed  ")) #Treks completed
z = int(input("Month Number  ")) #month
Rec_treks = Final_data.loc[(Final_data['no_of_days']<x)&(Final_data['Typical Season']==z)&(Final_data['difficulty']<=max(3,y+2))]
Mult = Rec_treks[['trek_name', 'no_of_days','region','difficulty', 'Distance_(in_km)',
       'Altitude_(in_mts)', 'price_(in_Rs)', 'SD', 'description', 'Extra_cost',
       'Single_tent', 'Backpack_offloading', 'Travel', 'Insurance',
       'Activity Name', 'No.of Days', 'Typical Season', 'Max. Altitude(M)',
       'BRS', 'Grade Classification']]
n=x
Multiple_treks=[]
for i in range (0,len(Mult)):
    for j in range(i+1, len(Mult)):
        if Mult.iloc[i][1]+Mult.iloc[j][1] < n-1:
            double = (Mult.iloc[i][0],Mult.iloc[j][0])
            Multiple_treks.append(double)
        for k in range(i+1, len(Mult)):
            if Mult.iloc[i][1]+Mult.iloc[j][1]+Mult.iloc[k][1]< n-2:
                triple = (Mult.iloc[i][0],Mult.iloc[j][0],Mult.iloc[k][0])
                Multiple_treks.append(triple)
for i in Multiple_treks:
    print(i)
Rec_treks

No of Days  15
Treks Completed  7
Month Number  12
('Nag Tibba Trek', 'Mukta Top Trek')
('Nag Tibba Trek', 'Dayara Bugyal Trek')
('Nag Tibba Trek', 'Kedarkantha Trek')
('Mukta Top Trek', 'Dayara Bugyal Trek')
('Mukta Top Trek', 'Kedarkantha Trek')
('Dayara Bugyal Trek', 'Kedarkantha Trek')


,trek_name,region,no_of_days,difficulty,Distance_(in_km),Altitude_(in_mts),price_(in_Rs),SD,description,Extra_cost,Single_tent,Backpack_offloading,Travel,Insurance,Activity Name,No.of Days,Typical Season,Max. Altitude(M),BRS,Grade Classification
0,Nag Tibba Trek,Uttrakhand,3.0,1,19,3004,3950,Pantwari to Pantwari,\nBrief Description \n\nIf you’re looking for ...,"['AddOns', 'Rs.0', 'Rs.600.00SingleAccommodati...",NaN,NaN,NaN,NaN,Nag Tibba Winter Trek,3,12,3000,1,Easy
50,Mukta Top Trek,Uttarakhand,6.0,2,28,3603,9600,Kuflon to Kuflon,\nBrief Description \n\n ...,"['AddOns', 'Rs.0', 'Rs.0', 'Rs.1750BackpackOff...",NaN,1750.0,Rs.2000Dehradun-Dehradun,248.0,Mukta Top Trek,6,12,3603,2,Easy
55,Dayara Bugyal Trek,Uttrakhand,5.0,2,21,3750,8000,Raithal to Raithal,\nBrief Description \n\n ...,"['AddOns', 'Rs.0DelhitoDehradun', 'Rs.0Dehradu...",NaN,918.0,Rs.2000Dehradun-Dehradun,206.0,Dayara Bugyal Trek,4,12,3000,1,Easy
57,Kedarkantha Trek,Uttarakhand,5.0,2,23,3810,8900,Sankri to Sankri,\nBrief Description \n\nLocated close to Govin...,"['AddOns', 'Rs.0', 'Rs.900.00SingleAccommodati...",90000.0,1224.0,Rs.2200Dehradun-Dehradun,206.0,Kedarkantha Trek,5,12,3810,2,Easy
